In [1]:
import os
from collections import Counter, defaultdict

import pandas as pd
import numpy as np
import jamo
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm

In [2]:
root_data_folder = "dataset/open/"

In [3]:
df = pd.read_csv(os.path.join(root_data_folder, 'train.csv'))

In [4]:
df['len'] = df['label'].str.len()

In [5]:
# # image resolution retrieval

# df_w, df_h = [], []
# for index, row in df.iterrows():
#     img_file = os.path.join(root_data_folder, row['img_path'])
#     img = cv2.imread(img_file)
#     h, w, c = img.shape
#     df_w.append(w), df_h.append(h)
# df['img_width'] = df_w
# df['img_height'] = df_h

In [6]:
df.head()

,id,img_path,label,len
0,TRAIN_00000,./train/TRAIN_00000.png,빨간색,3
1,TRAIN_00001,./train/TRAIN_00001.png,머,1
2,TRAIN_00002,./train/TRAIN_00002.png,차차,2
3,TRAIN_00003,./train/TRAIN_00003.png,써,1
4,TRAIN_00004,./train/TRAIN_00004.png,놓치다,3


In [7]:
corpus = set(df['label'].values)

In [8]:
df['len'].value_counts()

2    28631
1    23703
3    13514
4     9988
5     1026
6       26
Name: len, dtype: int64

In [9]:
len(df)

76888

In [10]:
len(corpus)

5628

In [11]:
df[df['len']==6]['label'].values

array(['불러일으키다', '불러일으키다', '불러일으키다', '불러일으키다', '두리번거리다', '두리번거리다',
       '두리번거리다', '두리번거리다', '불러일으키다', '불러일으키다', '불러일으키다', '불러일으키다',
       '불러일으키다', '불러일으키다', '두리번거리다', '불러일으키다', '불러일으키다', '불러일으키다',
       '두리번거리다', '두리번거리다', '두리번거리다', '두리번거리다', '불러일으키다', '두리번거리다',
       '두리번거리다', '두리번거리다'], dtype=object)

In [12]:
for i in range(1,7):
    data_with_length_i = df[df['len']==i]
    word_set = set(data_with_length_i['label'].values)
    distinct_scale = len(data_with_length_i)/len(word_set)
    print(f"Number of distinct label with length {i}: {len(word_set)} over {len(data_with_length_i)} ({distinct_scale:.1f}x)")

Number of distinct label with length 1: 2348 over 23703 (10.1x)
Number of distinct label with length 2: 1710 over 28631 (16.7x)
Number of distinct label with length 3: 847 over 13514 (16.0x)
Number of distinct label with length 4: 654 over 9988 (15.3x)
Number of distinct label with length 5: 67 over 1026 (15.3x)
Number of distinct label with length 6: 2 over 26 (13.0x)


In [13]:
# explore for more in depth labels
label_length = 5
data_with_length_i = df[df['len']==label_length]
# pd.set_option("display.max_rows", 100)
cnts = data_with_length_i['label'].value_counts()
print(cnts)

부러워하다    24
분명해지다    22
걱정스럽다    21
옛날이야기    20
내려다보다    20
         ..
흘러내리다    11
말씀드리다    11
들이마시다    11
바람직하다    11
찾아다니다    10
Name: label, Length: 67, dtype: int64


In [14]:
# checking the scale of the max-min occurence on each length
for i in range(1,7):
    data_with_length_i = df[df['len']==i]
    cnts = list(data_with_length_i['label'].value_counts())
    max_cnt, min_cnt = cnts[0], cnts[-1]
    range_scale = max_cnt/min_cnt
    print(f"Range of occurence for each word with length {i}: {min_cnt}-{max_cnt} ({range_scale:.1f}x)")

Range of occurence for each word with length 1: 1-106 (106.0x)
Range of occurence for each word with length 2: 9-61 (6.8x)
Range of occurence for each word with length 3: 6-38 (6.3x)
Range of occurence for each word with length 4: 8-31 (3.9x)
Range of occurence for each word with length 5: 10-24 (2.4x)
Range of occurence for each word with length 6: 12-14 (1.2x)


In [15]:
# display sample occurences of one word
def display_sample_of_word(word_input="활발해지다", columns=4, rows=5):
    samples = df[df['label']==word_input]
    imgs = samples['img_path'].values
    w = 15*len(word_input)
    h = 15
    fig = plt.figure(figsize=(8, 8))
    print(f"total occurence of '{word_input}': {len(imgs)}")
    for i in range(1, columns*rows +1):
        if i > len(imgs): break
        img_file = os.path.join(root_data_folder, imgs[i-1])
        img = cv2.imread(img_file)
        fig.add_subplot(rows, columns, i)
        plt.imshow(img)
        plt.axis('off')
    plt.show()
# display_sample_of_word()

In [16]:
# display_sample_of_word(word_input="부러워하다", columns=4, rows=6)

In [19]:
# # image resolution distribution
# for i in range(1,7):
#     data_with_length_i = df[df['len']==i]
#     img_width = list(data_with_length_i['img_width'])
#     img_height = list(data_with_length_i['img_height'])
#     max_w, min_w = max(img_width), min(img_width)
#     max_h, min_h = max(img_height), min(img_height)
#     print(f"Range of img_width for each word with length {i}: {min_w}-{max_w} ({max_w/min_w:.1f}x)")
#     print(f"Range of img_height for each word with length {i}: {min_h}-{max_h} ({max_h/min_h:.1f}x)")
#     print()

## Exploration on API

In [41]:
sample_pred = pd.read_csv('submission/sub_0.resnet50_rnn_AUGCombo_DataAdd_v1_sharpen_train_test.csv')
sample_pred.head()

,id,label
0,TEST_00000,날말
1,TEST_00001,상활
2,TEST_00002,받아들이다
3,TEST_00003,바구니
4,TEST_00004,살


In [21]:
# !pip install py-hangul-checker

In [22]:
from hangul_checker import KoreanSpellChecker

In [23]:
# --> refer to post_processing_1.py

# ksc = KoreanSpellChecker()
# hangul_checker_result = []
# for index, row in tqdm(sample_pred.iterrows(), total=sample_pred.shape[0]):
#     hangul_checker_result.append(ksc.check_spelling(row['label']).replace(" ", ""))
# sample_pred['hangul_checker'] = hangul_checker_result

In [24]:
hangul_checker_result = pd.read_csv('submission/sub_0.resnet50_rnn_AUGCombo_DataAdd_STR_v1_100_post-process1.csv')
sample_pred['hangul_checker'] = hangul_checker_result['label']

In [25]:
# !pip install symspellpy-ko

In [26]:
from symspellpy_ko import KoSymSpell, Verbosity

In [27]:
# --> refer to post_processing_2.py

# sym_spell = KoSymSpell()
# sym_spell.load_korean_dictionary(decompose_korean=True, load_bigrams=True)

# symspellpy_ko_result = []
# for index, row in tqdm(sample_pred.iterrows(), total=sample_pred.shape[0]):
#     for suggestion in sym_spell.lookup(row['label'], Verbosity.ALL):
#         symspellpy_ko_result.append(suggestion.term)
#         break
# sample_pred['symspellpy_ko'] = symspellpy_ko_result

In [45]:
symspellpy_ko_pred = pd.read_csv('submission/sub_0.resnet50_rnn_AUGCombo_DataAdd_v1_sharpen_train_test_post-process2.csv')
sample_pred['symspellpy_ko'] = symspellpy_ko_pred['label']

In [29]:
# def remove_whitespace_and_replace(filename):
#     df = pd.read_csv(filename)
#     clean_label = []
#     for index, row in df.iterrows():
#         clean_label.append(row['label'].replace(" ", ""))
#     df['label'] = clean_label
#     df.to_csv(filename, index=False)
    
# remove_whitespace_and_replace('submission/sub_0.resnet50_rnn_AUGCombo_DataAdd_STR_v1_100_post-process1.csv')

In [35]:
p1 = "hangul_checker"
p2 = "symspellpy_ko"
print(f"index | label | {p1} | {p2}")
p1rev, p2rev, p1p2rev, p1p2revdiff = 0, 0, 0, 0
p1p2revsample = []
for index, row in sample_pred.iterrows():
    if row['label'] != row[p1] or row['label'] != row[p2]:
        if row['label'] != row[p1]: p1rev += 1
        if row['label'] != row[p2]: p2rev += 1
        if row['label'] != row[p1] and row['label'] != row[p2]:
            if row[p1] != row[p2]: p1p2revdiff += 1
            p1p2rev += 1
            p1p2revsample.append(row)
        print(f"[{row['id']}] | {row['label']} | {row[p1]} | {row[p2]}")

print(f"Total data: {len(sample_pred)}")
print(f"Total revision by {p1}: {p1rev}")
print(f"Total revision by {p2}: {p2rev}")
print(f"Total revision by {p1} and {p2}: {p1p2rev}")
print(f"Total revision by {p1} and {p2} which are different: {p1p2revdiff}")

index | label | hangul_checker | symspellpy_ko
[TEST_00000] | 남말 | 남말 | 남자
[TEST_00001] | 상활 | 상활 | 사흘
[TEST_00003] | 바구니 | 바구니 | 바가지
[TEST_00007] | 센터 | 센터 | 센
[TEST_00009] | 광주 | 광주 | 경주
[TEST_00010] | 나나 | 나나 | 난
[TEST_00012] | 도도 | 도도 | 도로
[TEST_00013] | 슬표 | 슬포 | 실패
[TEST_00018] | 흔자 | 은자 | 환자
[TEST_00022] | 피우다 | 피우다 | 싸우다
[TEST_00025] | 겨울 | 겨울 | 겨우
[TEST_00026] | 예선 | 예선 | 예산
[TEST_00033] | 사진기 | 사진기 | 사진
[TEST_00034] | 내리다 | 내리다 | 놀다
[TEST_00035] | 평명 | 평명 | 명령
[TEST_00036] | 맛없다 | 맛없다 | 맛있다
[TEST_00038] | 우선 | 우선 | 어떤
[TEST_00039] | 대답 | 대답 | 대사
[TEST_00040] | 학생 | 학생 | 항상
[TEST_00042] | 본질 | 본질 | 본인
[TEST_00043] | 현대 | 현대 | 현재
[TEST_00044] | 속하다 | 속하다 | 착하다
[TEST_00045] | 지저문하다 | 지저분하다 | 지저문하다
[TEST_00046] | 불다 | 불다 | 보다
[TEST_00047] | 아래 | 아래 | 알
[TEST_00051] | 여속 | 여속 | 계속
[TEST_00054] | 기차 | 기차 | 기사
[TEST_00055] | 저것 | 저것 | 것
[TEST_00056] | 기침 | 기침 | 아침
[TEST_00057] | 의쪽 | 위쪽 | 위쪽
[TEST_00061] | 학비 | 학비 | 학기
[TEST_00066] | 경검 | 경건 | 경험
[TEST_00068] | 역사상 | 역사상 | 역사가
[TEST

[TEST_02263] | 수업이 | 수업이 | 수업
[TEST_02264] | 일생 | 일생 | 인생
[TEST_02265] | 환갑 | 환갑 | 환자
[TEST_02267] | 만부 | 만부 | 만두
[TEST_02268] | 오해 | 오해 | 이해
[TEST_02270] | 내주다 | 내주다 | 내다
[TEST_02272] | 그렌데 | 그렌데 | 근데
[TEST_02273] | 국내외 | 국내외 | 국내
[TEST_02277] | 다툼 | 다툼 | 다음
[TEST_02278] | 여름 | 여름 | 이름
[TEST_02279] | 선생 | 선생 | 인생
[TEST_02280] | 명명 | 명명 | 명령
[TEST_02282] | 소망 | 소망 | 사망
[TEST_02288] | 파다 | 파다 | 바다
[TEST_02289] | 시외 | 시외 | 시야
[TEST_02292] | 불편 | 불편 | 우편
[TEST_02293] | 소개 | 소개 | 속
[TEST_02296] | 사사 | 사사 | 사자
[TEST_02297] | 소식 | 소식 | 사실
[TEST_02299] | 아버님 | 아버님 | 아버지
[TEST_02307] | 육십 | 육십 | 욕실
[TEST_02308] | 푸다 | 푸다 | 다
[TEST_02309] | 오후 | 오후 | 이후
[TEST_02310] | 그렇지 | 그렇지 | 그렇게
[TEST_02312] | 구십 | 구십 | 수십
[TEST_02314] | 시점 | 시점 | 시험
[TEST_02315] | 매력 | 매력 | 새벽
[TEST_02318] | 서다 | 서다 | 사다
[TEST_02320] | 발목 | 발목 | 바로
[TEST_02321] | 넓다 | 넓다 | 낫다
[TEST_02324] | 중영하다 | 중요하다 | 중영하다
[TEST_02325] | 찻잔 | 찻잔 | 한잔
[TEST_02326] | 다수 | 다수 | 다시
[TEST_02328] | 분면 | 분면 | 분명
[TEST_02331] | 불고 | 불고 | 바로
[T

[TEST_04340] | 잎 | 잎 | 이
[TEST_04341] | 그날 | 그날 | 그냥
[TEST_04342] | 밤색 | 밤색 | 발생
[TEST_04343] | 소핑 | 쇼핑 | 쇼핑
[TEST_04345] | 먹울하다 | 먹을하다 | 먹울하다
[TEST_04346] | 불과 | 불과 | 불
[TEST_04348] | 하천 | 하천 | 한
[TEST_04350] | 얼줄 | 올줄 | 얼줄
[TEST_04351] | 세탁기 | 세탁기 | 세탁
[TEST_04352] | 길다 | 길다 | 있다
[TEST_04355] | 동물 | 동물 | 도움
[TEST_04356] | 세다 | 세다 | 사다
[TEST_04358] | 나중 | 나중 | 아주
[TEST_04359] | 영국 | 영국 | 연구
[TEST_04361] | 조용히 | 조용히 | 종이
[TEST_04362] | 나서다 | 나서다 | 낫다
[TEST_04363] | 목물 | 목물 | 목숨
[TEST_04364] | 약혼자 | 약혼자 | 약혼녀
[TEST_04368] | 술집 | 술집 | 수집
[TEST_04371] | 외할머니 | 외할머니 | 할머니
[TEST_04374] | 통장 | 통장 | 통증
[TEST_04379] | 경명 | 경명 | 명령
[TEST_04380] | 늦다 | 늦다 | 맞다
[TEST_04381] | 의의 | 의의 | 예의
[TEST_04390] | 명명 | 명명 | 명령
[TEST_04391] | 첫물 | 첫물 | 건물
[TEST_04394] | 구십 | 구십 | 수십
[TEST_04397] | 긴장하다 | 긴장하다 | 굉장하다
[TEST_04398] | 시월 | 시월 | 세월
[TEST_04399] | 연세 | 연세 | 언제
[TEST_04400] | 장래 | 장래 | 잘
[TEST_04401] | 의욕 | 의욕 | 예약
[TEST_04403] | 경치 | 경치 | 경기
[TEST_04405] | 원하다 | 원하다 | 일하다
[TEST_04408] | 음동 | 음동 | 

[TEST_06742] | 젓다 | 젓다 | 절대
[TEST_06743] | 배개 | 베개 | 백
[TEST_06748] | 강남 | 강남 | 장난
[TEST_06751] | 허허 | 허허 | 어서
[TEST_06752] | 연설 | 연설 | 여성
[TEST_06753] | 탁구 | 탁구 | 특수
[TEST_06757] | 전전 | 전전 | 전선
[TEST_06758] | 지키다 | 지키다 | 지내다
[TEST_06762] | 반번 | 만번 | 한번
[TEST_06765] | 하실 | 하실 | 사실
[TEST_06767] | 상상 | 상상 | 항상
[TEST_06768] | 피해 | 피해 | 이해
[TEST_06771] | 창구 | 창구 | 창고
[TEST_06774] | 여간 | 여간 | 야간
[TEST_06777] | 한복 | 한복 | 한번
[TEST_06779] | 그불다 | 그불타 | 그불다
[TEST_06780] | 불편 | 불편 | 우편
[TEST_06781] | 사십 | 사십 | 사실
[TEST_06790] | 암껏 | 맘껏 | 힘껏
[TEST_06792] | 정치학 | 정치학 | 정치적
[TEST_06793] | 필요 | 필요 | 필
[TEST_06795] | 삼월 | 삼월 | 세월
[TEST_06797] | 건강 | 건강 | 긴장
[TEST_06804] | 활기 | 활기 | 헬기
[TEST_06805] | 안되다 | 안되다 | 안다
[TEST_06807] | 냉면 | 냉면 | 생명
[TEST_06808] | 직원 | 직원 | 지원
[TEST_06812] | 적다 | 적다 | 먹다
[TEST_06813] | 갈색 | 갈색 | 발생
[TEST_06815] | 축하 | 축하 | 추가
[TEST_06817] | 일정 | 일정 | 일종
[TEST_06818] | 볶다 | 볶다 | 보다
[TEST_06820] | 없이 | 없이 | 어디
[TEST_06821] | 최고급 | 최고급 | 고급
[TEST_06822] | 지출 | 지출 | 진출
[TEST_068

[TEST_09206] | 야하다 | 야하다 | 하다
[TEST_09207] | 우표 | 우표 | 투표
[TEST_09209] | 앞시하다 | 앞에하다 | 앞시하다
[TEST_09211] | 넓다 | 넓다 | 낫다
[TEST_09212] | 활영하다 | 활용하다 | 활영하다
[TEST_09213] | 반반 | 반반 | 반찬
[TEST_09215] | 분리 | 분리 | 우리
[TEST_09219] | 칠십 | 칠십 | 출신
[TEST_09222] | 일무일 | 일무일 | 일주일
[TEST_09223] | 사무소 | 사무소 | 산소
[TEST_09224] | 참다 | 참다 | 차마
[TEST_09225] | 특별 | 특별 | 특별히
[TEST_09227] | 뜨럽다 | 스럽다 | 그러다
[TEST_09230] | 여화 | 여화 | 영화
[TEST_09232] | 일른 | 이른 | 얼른
[TEST_09233] | 진하다 | 진하다 | 일하다
[TEST_09235] | 심리 | 심리 | 이미
[TEST_09236] | 발길 | 발길 | 달걀
[TEST_09240] | 맑다 | 맑다 | 맞다
[TEST_09241] | 기분 | 기분 | 그분
[TEST_09242] | 품경 | 풍경 | 풍경
[TEST_09245] | 게임 | 게임 | 개인
[TEST_09247] | 미인 | 미인 | 시인
[TEST_09248] | 정기 | 정기 | 저기
[TEST_09249] | 만약 | 만약 | 마약
[TEST_09251] | 서쪽 | 서쪽 | 서로
[TEST_09253] | 역사 | 역사 | 여자
[TEST_09256] | 하순 | 하순 | 한
[TEST_09257] | 이외 | 이외 | 이유
[TEST_09258] | 노랗다 | 노랗다 | 놀다
[TEST_09260] | 대다 | 대다 | 대사
[TEST_09262] | 고요부 | 고요부 | 공부
[TEST_09265] | 거실 | 거실 | 것
[TEST_09266] | 늦가울 | 늦가을 | 늦가울
[TEST_09269] | 얼마

...

[TEST_63989] | 퇴지 | 퇴직 | 가지
[TEST_63993] | 마시다 | 마시다 | 맞다
[TEST_63994] | 국제화 | 국제화 | 국제
[TEST_63995] | 용도 | 용도 | 용돈
[TEST_64002] | 막상 | 막상 | 항상
[TEST_64009] | 아기 | 아기 | 아니
[TEST_64014] | 실필 | 실실 | 실패
[TEST_64015] | 변화 | 변화 | 전화
[TEST_64017] | 피우다 | 피우다 | 싸우다
[TEST_64019] | 적다 | 적다 | 먹다
[TEST_64020] | 예대 | 예대 | 어디
[TEST_64022] | 봉지 | 봉지 | 방지
[TEST_64026] | 욕심 | 욕심 | 욕실
[TEST_64028] | 점심 | 점심 | 잠시
[TEST_64030] | 공항 | 공항 | 고장
[TEST_64031] | 불관 | 불관 | 물건
[TEST_64033] | 방안 | 방안 | 장난
[TEST_64035] | 도도 | 도도 | 도로
[TEST_64037] | 결석 | 결석 | 녀석
[TEST_64038] | 전전 | 전전 | 전선
[TEST_64039] | 사과하다 | 사과하다 | 착하다
[TEST_64044] | 첫방 | 첫방 | 첫날
[TEST_64046] | 퇴지고기 | 퇴직고기 | 퇴지고기
[TEST_64048] | 담당자 | 담당자 | 담당
[TEST_64051] | 간식 | 간식 | 관심
[TEST_64054] | 남남 | 남남 | 남자
[TEST_64056] | 보람 | 보람 | 바람
[TEST_64057] | 인간 | 인간 | 민간
[TEST_64059] | 삼다 | 삼다 | 사다
[TEST_64064] | 활단보도 | 활달보도 | 활단보도
[TEST_64067] | 때문 | 때문 | 문
[TEST_64068] | 창조 | 창조 | 창고
[TEST_64073] | 예방 | 예방 | 이상
[TEST_64074] | 참피하다 | 창피하다 | 참피하다
[TEST_64079] | 서류

[TEST_65797] | 걸다 | 걸다 | 걷다
[TEST_65800] | 짜다 | 짜다 | 바다
[TEST_65801] | 그리고 | 그리고 | 그리로
[TEST_65802] | 대신하다 | 대신하다 | 대단하다
[TEST_65807] | 박다 | 박다 | 바다
[TEST_65815] | 후배 | 후배 | 해
[TEST_65816] | 둥들다 | 둥글다 | 둥들다
[TEST_65817] | 이용 | 이용 | 비용
[TEST_65818] | 너우 | 너우 | 넌
[TEST_65822] | 싸다 | 싸다 | 바다
[TEST_65826] | 원하다 | 원하다 | 일하다
[TEST_65827] | 둥들다 | 둥글다 | 둥들다
[TEST_65830] | 늦다 | 늦다 | 맞다
[TEST_65831] | 파란색 | 파란색 | 빨간색
[TEST_65832] | 벗다 | 벗다 | 보다
[TEST_65833] | 선장 | 선장 | 천장
[TEST_65838] | 논쟁 | 논쟁 | 전쟁
[TEST_65841] | 비비 | 비비 | 이미
[TEST_65844] | 성인 | 성인 | 정신
[TEST_65845] | 만족 | 만족 | 가족
[TEST_65846] | 외국 | 외국 | 미국
[TEST_65848] | 언재나 | 언제나 | 언젠가
[TEST_65849] | 비누 | 비누 | 빈
[TEST_65853] | 콜라 | 콜라 | 몰래
[TEST_65855] | 질심시간 | 질심시간 | 점심시간
[TEST_65856] | 굶다 | 굶다 | 같다
[TEST_65857] | 장미 | 장미 | 자기
[TEST_65858] | 그늘 | 그늘 | 그냥
[TEST_65861] | 대응 | 대응 | 응
[TEST_65867] | 평명 | 평명 | 명령
[TEST_65869] | 종통 | 종통 | 보통
[TEST_65871] | 비비 | 비비 | 이미
[TEST_65874] | 이용 | 이용 | 비용
[TEST_65875] | 너무 | 너무 | 나무
[TEST_65878] | 한낮 | 한낮

[TEST_67983] | 회원 | 회원 | 퇴원
[TEST_67986] | 슬펴하다 | 슬퍼하다 | 슬펴하다
[TEST_67987] | 보험 | 보험 | 시험
[TEST_67988] | 오이 | 오이 | 아이
[TEST_67989] | 채우다 | 채우다 | 싸우다
[TEST_67990] | 일체 | 일체 | 이제
[TEST_67995] | 가난 | 가난 | 가만
[TEST_67998] | 주차장 | 주차장 | 주장
[TEST_67999] | 수석 | 수석 | 구석
[TEST_68000] | 비다 | 비다 | 보다
[TEST_68001] | 무더위 | 무더위 | 무덤
[TEST_68002] | 색깥 | 색같 | 색깥
[TEST_68003] | 취미 | 취미 | 이미
[TEST_68005] | 아들 | 아들 | 오늘
[TEST_68006] | 군민 | 군민 | 국민
[TEST_68007] | 요일 | 요일 | 일
[TEST_68008] | 군형 | 군형 | 분명
[TEST_68011] | 수필 | 수필 | 사실
[TEST_68014] | 치마 | 치마 | 차마
[TEST_68015] | 박수 | 박수 | 박스
[TEST_68016] | 말다 | 말다 | 맞다
[TEST_68017] | 무집다 | 무지다 | 무집다
[TEST_68018] | 봉사 | 봉사 | 복사
[TEST_68020] | 학생 | 학생 | 항상
[TEST_68023] | 해당 | 해당 | 행동
[TEST_68024] | 버리다 | 버리다 | 저러다
[TEST_68028] | 담요 | 담요 | 단어
[TEST_68031] | 씨찌 | 씨지 | 이미
[TEST_68036] | 그로 | 그로 | 그래
[TEST_68037] | 박수 | 박수 | 박스
[TEST_68039] | 명히 | 명이 | 영화
[TEST_68040] | 약속 | 약속 | 양복
[TEST_68041] | 축하 | 축하 | 추가
[TEST_68046] | 노완색 | 노와는색 | 노완색
[TEST_68048] | 묻다 | 묻다 | 맞

[TEST_70159] | 찾다 | 찾다 | 맞다
[TEST_70162] | 한식 | 한식 | 현실
[TEST_70164] | 종로 | 종로 | 종류
[TEST_70165] | 주변 | 주변 | 준
[TEST_70167] | 식탁 | 식탁 | 시각
[TEST_70171] | 인삼 | 인삼 | 인사
[TEST_70175] | 몸통 | 몸통 | 보통
[TEST_70176] | 대책 | 대책 | 대체
[TEST_70177] | 파파 | 파파 | 하나
[TEST_70179] | 다섯 | 다섯 | 다시
[TEST_70182] | 쭉 | 쭉 | 쪽
[TEST_70183] | 해군 | 해군 | 군
[TEST_70185] | 허가 | 허가 | 화가
[TEST_70186] | 어떻다 | 어떻다 | 어쩌다
[TEST_70187] | 가치관 | 가치관 | 기관
[TEST_70189] | 미비 | 미비 | 미리
[TEST_70190] | 싸다 | 싸다 | 바다
[TEST_70197] | 식탁 | 식탁 | 시각
[TEST_70198] | 슬페하다 | 슬퍼하다 | 슬페하다
[TEST_70201] | 사사 | 사사 | 사자
[TEST_70202] | 나라 | 나라 | 날
[TEST_70207] | 비디오 | 비디오 | 라디오
[TEST_70208] | 아프다 | 아프다 | 안다
[TEST_70209] | 발발 | 발발 | 바람
[TEST_70212] | 흐르다 | 흐르다 | 그러다
[TEST_70213] | 귀국 | 귀국 | 미국
[TEST_70214] | 파파 | 파파 | 하나
[TEST_70217] | 진리 | 진리 | 진료
[TEST_70219] | 보출하다 | 보철하다 | 보출하다
[TEST_70220] | 남쪽 | 남쪽 | 깜짝
[TEST_70222] | 덥다 | 덥다 | 쉽다
[TEST_70226] | 비서 | 비서 | 부서
[TEST_70228] | 정정 | 정정 | 걱정
[TEST_70230] | 렌즈 | 렌즈 | 렌
[TEST_70232] | 의류 | 의류 | 우리
[T

[TEST_72269] | 메모 | 메모 | 몸
[TEST_72276] | 학급 | 학급 | 한글
[TEST_72279] | 고생 | 고생 | 곳
[TEST_72280] | 구성 | 구성 | 구멍
[TEST_72281] | 정정 | 정정 | 걱정
[TEST_72282] | 발견 | 발견 | 발전
[TEST_72286] | 운문 | 운문 | 운전
[TEST_72288] | 총장 | 총장 | 고장
[TEST_72295] | 다섯 | 다섯 | 다시
[TEST_72297] | 실태 | 실태 | 실패
[TEST_72299] | 사방 | 사방 | 사랑
[TEST_72303] | 밥다 | 바다 | 바다
[TEST_72304] | 몸살 | 몸살 | 목사
[TEST_72305] | 생산자 | 생산자 | 생산
[TEST_72307] | 알맛다 | 알맞다 | 알맛다
[TEST_72308] | 모자 | 모자 | 못
[TEST_72313] | 아기 | 아기 | 아니
[TEST_72320] | 놓다 | 놓다 | 좋다
[TEST_72323] | 홍차 | 홍차 | 공짜
[TEST_72324] | 만점 | 만점 | 만큼
[TEST_72325] | 마하 | 마하 | 아하
[TEST_72327] | 볶다 | 볶다 | 보다
[TEST_72328] | 여권 | 여권 | 원
[TEST_72330] | 혼자 | 혼자 | 환자
[TEST_72332] | 한복 | 한복 | 한번
[TEST_72335] | 아기 | 아기 | 아니
[TEST_72339] | 가계 | 가계 | 각
[TEST_72342] | 전화기 | 전화기 | 전화
[TEST_72345] | 빛깥 | 빛같 | 바깥
[TEST_72348] | 교육 | 교육 | 기억
[TEST_72349] | 재우다 | 재우다 | 싸우다
[TEST_72353] | 유행 | 유행 | 여행
[TEST_72355] | 해설 | 해설 | 해석
[TEST_72358] | 보이다 | 보이다 | 보다
[TEST_72359] | 구월 | 구월 | 개월
[TEST_72360] 

In [37]:
p1p2revsample

[id                TEST_00013
 label                     슬표
 hangul_checker            슬포
 symspellpy_ko             실패
 Name: 13, dtype: object,
 id                TEST_00018
 label                     흔자
 hangul_checker            은자
 symspellpy_ko             환자
 Name: 18, dtype: object,
 id                TEST_00057
 label                     의쪽
 hangul_checker            위쪽
 symspellpy_ko             위쪽
 Name: 57, dtype: object,
 id                TEST_00066
 label                     경검
 hangul_checker            경건
 symspellpy_ko             경험
 Name: 66, dtype: object,
 id                TEST_00133
 label                     물프
 hangul_checker            물포
 symspellpy_ko              물
 Name: 133, dtype: object,
 id                TEST_00134
 label                     처별
 hangul_checker            처벌
 symspellpy_ko             차별
 Name: 134, dtype: object,
 id                TEST_00138
 label                     학간
 hangul_checker            항간
 symspellpy_ko             약간
 N

In [46]:
# checking performance in the corpus after the revision

# p = "hangul_checker"
p = "symspellpy_ko"
print(f"index | label | {p}")
diff, label_in_corpus, p_in_corpus, both_in_corpus = 0, 0, 0, 0
for index, row in sample_pred.iterrows():
    if row['label'] != row[p]:
        diff += 1
        if row['label'] in corpus: label_in_corpus += 1
        if row[p] in corpus: p_in_corpus += 1
        if row['label'] in corpus and row[p] in corpus: both_in_corpus += 1
        print(f"[{row['id']}] | {row['label']} | {row[p]}")

print(f"Total revision using '{p}': {diff}/{len(sample_pred)} ({diff/len(sample_pred)*100:.2f}%)")
print(f"Over the revised prediction, label has {label_in_corpus}/{diff} ({label_in_corpus/diff*100:.2f}%) in the corpus")
print(f"Over the revised prediction, '{p}' has {p_in_corpus}/{diff} ({p_in_corpus/diff*100:.2f}%) in the corpus")
print(f"The ambiguity because of both label and '{p}' is in the corpus is {both_in_corpus}/{diff} ({both_in_corpus/diff*100:.2f}%)")

index | label | symspellpy_ko
[TEST_00000] | 날말 | 날짜
[TEST_00001] | 상활 | 사흘
[TEST_00003] | 바구니 | 바가지
[TEST_00009] | 광주 | 경주
[TEST_00012] | 도도 | 도로
[TEST_00018] | 혼자 | 환자
[TEST_00025] | 겨울 | 겨우
[TEST_00026] | 예선 | 예산
[TEST_00035] | 평평 | 명령
[TEST_00036] | 맛없다 | 맛있다
[TEST_00038] | 우선 | 어떤
[TEST_00039] | 대답 | 대사
[TEST_00040] | 학생 | 항상
[TEST_00043] | 현대 | 현재
[TEST_00044] | 속하다 | 착하다
[TEST_00046] | 불다 | 보다
[TEST_00051] | 예속 | 계속
[TEST_00054] | 기차 | 기사
[TEST_00056] | 기침 | 아침
[TEST_00066] | 경경 | 공격
[TEST_00068] | 역사상 | 역사가
[TEST_00075] | 사심 | 사실
[TEST_00076] | 선물 | 건물
[TEST_00077] | 아직 | 오직
[TEST_00080] | 이자 | 이제
[TEST_00083] | 소망 | 사망
[TEST_00087] | 마마 | 아마
[TEST_00094] | 버나나 | 바나나
[TEST_00095] | 경우 | 겨우
[TEST_00096] | 마지막 | 하지만
[TEST_00097] | 영복 | 양복
[TEST_00098] | 소리 | 꼬리
[TEST_00104] | 체험하다 | 위험하다
[TEST_00110] | 도도 | 도로
[TEST_00116] | 반반 | 반찬
[TEST_00117] | 논문 | 전문
[TEST_00132] | 다전 | 사전
[TEST_00134] | 처별 | 차별
[TEST_00136] | 서다 | 사다
[TEST_00138] | 선반 | 순간
[TEST_00143] | 보험 | 시험
[TEST_00145

[TEST_03030] | 흔자 | 환자
[TEST_03032] | 씨우다 | 싸우다
[TEST_03035] | 출종 | 출장
[TEST_03036] | 진통 | 전통
[TEST_03037] | 최고 | 최소
[TEST_03040] | 염어 | 영어
[TEST_03043] | 반반 | 반찬
[TEST_03045] | 이것 | 이거
[TEST_03046] | 섞다 | 사다
[TEST_03048] | 전화하다 | 정확하다
[TEST_03049] | 똑하다 | 착하다
[TEST_03050] | 사투리 | 아무리
[TEST_03055] | 파파 | 하나
[TEST_03068] | 일식 | 일찍
[TEST_03069] | 굽다 | 같다
[TEST_03071] | 냄새 | 냄비
[TEST_03076] | 걸다 | 걷다
[TEST_03078] | 관람 | 관심
[TEST_03082] | 반깝다 | 가깝다
[TEST_03086] | 이술 | 기술
[TEST_03087] | 어머님 | 어머니
[TEST_03089] | 발발 | 바람
[TEST_03092] | 신문 | 신분
[TEST_03093] | 가슴 | 가끔
[TEST_03095] | 육십 | 욕실
[TEST_03098] | 신라 | 신사
[TEST_03102] | 학교 | 학기
[TEST_03106] | 이용 | 비용
[TEST_03109] | 물물 | 눈물
[TEST_03112] | 뺐다 | 있다
[TEST_03113] | 찻째 | 첫째
[TEST_03116] | 아니요 | 아니야
[TEST_03121] | 희망 | 사망
[TEST_03126] | 주식 | 지식
[TEST_03130] | 사투리 | 아무리
[TEST_03136] | 느리다 | 그러다
[TEST_03145] | 유영 | 여왕
[TEST_03149] | 전국 | 전부
[TEST_03150] | 공개 | 공기
[TEST_03152] | 물물 | 눈물
[TEST_03157] | 자꾸 | 자주
[TEST_03158] | 향수 | 형수
[TEST_03160] |

[TEST_06004] | 암껏 | 힘껏
[TEST_06010] | 전국 | 전부
[TEST_06012] | 된장 | 긴장
[TEST_06016] | 병원 | 공원
[TEST_06021] | 희망 | 사망
[TEST_06025] | 연인 | 여인
[TEST_06026] | 발옥 | 바로
[TEST_06027] | 재산 | 대신
[TEST_06032] | 학교 | 학기
[TEST_06034] | 번지 | 먼지
[TEST_06036] | 파티 | 파리
[TEST_06041] | 장남 | 장난
[TEST_06042] | 붙다 | 보다
[TEST_06045] | 진행 | 인생
[TEST_06046] | 유행 | 여행
[TEST_06049] | 그릇 | 그래
[TEST_06050] | 장비 | 낭비
[TEST_06053] | 커지다 | 멋지다
[TEST_06054] | 등산 | 당신
[TEST_06055] | 결정 | 열정
[TEST_06058] | 매우 | 매주
[TEST_06059] | 모집 | 고집
[TEST_06060] | 벌다 | 멀다
[TEST_06061] | 금종 | 금지
[TEST_06071] | 승리 | 그리
[TEST_06075] | 마들 | 모든
[TEST_06082] | 파란색 | 빨간색
[TEST_06086] | 부품 | 부분
[TEST_06089] | 것속 | 계속
[TEST_06092] | 법적 | 성적
[TEST_06093] | 마음 | 다음
[TEST_06094] | 키스 | 미스
[TEST_06096] | 보전 | 오전
[TEST_06099] | 새다 | 사다
[TEST_06100] | 범위 | 범죄
[TEST_06103] | 아빠 | 아마
[TEST_06105] | 아가 | 아마
[TEST_06106] | 씻다 | 있다
[TEST_06107] | 오빠 | 오다
[TEST_06109] | 겨울 | 겨우
[TEST_06116] | 형부 | 형수
[TEST_06121] | 미움 | 미국
[TEST_06123] | 신발 | 신사
[TEST_0

[TEST_08943] | 일식 | 일찍
[TEST_08949] | 문재 | 문제
[TEST_08953] | 봉지 | 방지
[TEST_08955] | 질적 | 지적
[TEST_08959] | 면면 | 몇몇
[TEST_08969] | 사십 | 사실
[TEST_08972] | 학습 | 박스
[TEST_08973] | 찌찌 | 이미
[TEST_08983] | 발견 | 발전
[TEST_08984] | 평평 | 명령
[TEST_08986] | 한자 | 하나
[TEST_08988] | 만약 | 마약
[TEST_08989] | 웃다 | 있다
[TEST_08990] | 핵시 | 혹시
[TEST_08991] | 둘쩍 | 둘째
[TEST_08993] | 동불 | 동부
[TEST_08999] | 킬로 | 실로
[TEST_09010] | 들르다 | 다르다
[TEST_09013] | 신발 | 신사
[TEST_09017] | 조상 | 조사
[TEST_09018] | 비디오 | 라디오
[TEST_09020] | 독돈 | 속도
[TEST_09027] | 호남 | 하나
[TEST_09032] | 생활 | 생일
[TEST_09036] | 발끝 | 바깥
[TEST_09041] | 봉사 | 복사
[TEST_09044] | 아직 | 오직
[TEST_09046] | 신고 | 신호
[TEST_09047] | 많다 | 맞다
[TEST_09048] | 교실 | 사실
[TEST_09051] | 최신 | 대신
[TEST_09052] | 진단 | 집단
[TEST_09057] | 신다 | 신사
[TEST_09062] | 몰라 | 몰래
[TEST_09063] | 일곱 | 일본
[TEST_09064] | 방금 | 다음
[TEST_09067] | 살인 | 사진
[TEST_09069] | 악기 | 학기
[TEST_09071] | 청소 | 처음
[TEST_09074] | 섞다 | 사다
[TEST_09080] | 점심 | 잠시
[TEST_09087] | 간접 | 안전
[TEST_09088] | 왕비 | 아이
[TEST_0

[TEST_11848] | 파파 | 하나
[TEST_11850] | 손가락 | 손바닥
[TEST_11851] | 운수 | 순수
[TEST_11856] | 절차 | 점차
[TEST_11857] | 회사 | 하나
[TEST_11861] | 농담 | 상담
[TEST_11866] | 공주 | 공부
[TEST_11869] | 연습 | 연극
[TEST_11872] | 점심 | 잠시
[TEST_11873] | 반번 | 한번
[TEST_11874] | 바마 | 아마
[TEST_11879] | 도도 | 도로
[TEST_11880] | 변경 | 변명
[TEST_11888] | 음반 | 그만
[TEST_11894] | 비째 | 번째
[TEST_11899] | 까닭 | 바다
[TEST_11900] | 마음 | 다음
[TEST_11902] | 가득하다 | 가능하다
[TEST_11903] | 가능 | 기능
[TEST_11907] | 다른 | 다음
[TEST_11913] | 중국 | 중간
[TEST_11918] | 파파 | 하나
[TEST_11920] | 교환 | 교훈
[TEST_11922] | 확보 | 화가
[TEST_11935] | 키스 | 미스
[TEST_11936] | 관깝다 | 가깝다
[TEST_11937] | 그렇다 | 그러다
[TEST_11938] | 울다 | 우리
[TEST_11941] | 입다 | 있다
[TEST_11942] | 간접 | 안전
[TEST_11944] | 여칠 | 며칠
[TEST_11948] | 총각 | 생각
[TEST_11949] | 이것 | 이거
[TEST_11958] | 인도 | 온도
[TEST_11960] | 게임 | 개인
[TEST_11961] | 일급 | 월급
[TEST_11963] | 시작 | 시장
[TEST_11968] | 학습 | 박스
[TEST_11969] | 고기 | 거기
[TEST_11970] | 평평 | 명령
[TEST_11974] | 신경 | 안경
[TEST_11981] | 책상 | 대강
[TEST_11983] | 햇물 | 생물
[

...

[TEST_60850] | 실현 | 실은
[TEST_60862] | 중순 | 정신
[TEST_60864] | 정장 | 정상
[TEST_60875] | 장미 | 자기
[TEST_60876] | 농부 | 공부
[TEST_60887] | 강하다 | 착하다
[TEST_60891] | 업다 | 어디
[TEST_60894] | 호선 | 조선
[TEST_60895] | 시월 | 세월
[TEST_60897] | 삼월 | 세월
[TEST_60898] | 운문 | 운전
[TEST_60899] | 후석 | 해석
[TEST_60900] | 피디 | 어디
[TEST_60903] | 정정 | 걱정
[TEST_60910] | 시월 | 세월
[TEST_60920] | 오축 | 오직
[TEST_60921] | 웃기다 | 여기다
[TEST_60922] | 그릇 | 그래
[TEST_60926] | 콜라 | 몰래
[TEST_60938] | 친하다 | 착하다
[TEST_60939] | 초밥 | 초반
[TEST_60943] | 전간 | 순간
[TEST_60961] | 실습 | 실은
[TEST_60962] | 식품 | 작품
[TEST_60966] | 일립 | 이리
[TEST_60967] | 저자 | 제자
[TEST_60968] | 염마 | 아마
[TEST_60972] | 거액 | 거의
[TEST_60973] | 녹음 | 조금
[TEST_60977] | 발발 | 바람
[TEST_60978] | 제일 | 메일
[TEST_60988] | 테이프 | 데이트
[TEST_60999] | 상하다 | 착하다
[TEST_61009] | 찌찌 | 이미
[TEST_61011] | 축하 | 추가
[TEST_61015] | 일식 | 일찍
[TEST_61017] | 월세 | 열쇠
[TEST_61022] | 아깝다 | 가깝다
[TEST_61024] | 한자 | 하나
[TEST_61025] | 전화하다 | 정확하다
[TEST_61026] | 전구 | 전부
[TEST_61027] | 국사 | 국가
[TEST_61028] | 성함 

[TEST_63690] | 굳이 | 같이
[TEST_63705] | 남쪽 | 깜짝
[TEST_63710] | 맑다 | 맞다
[TEST_63716] | 아흔 | 마흔
[TEST_63717] | 발발 | 바람
[TEST_63720] | 이번 | 어떤
[TEST_63728] | 술집 | 수집
[TEST_63734] | 유능하다 | 가능하다
[TEST_63735] | 도중 | 도장
[TEST_63738] | 씻다 | 있다
[TEST_63739] | 아흔 | 마흔
[TEST_63741] | 애기 | 얘기
[TEST_63748] | 갈비 | 같이
[TEST_63755] | 축소 | 숙소
[TEST_63760] | 초점 | 오전
[TEST_63769] | 책상 | 대강
[TEST_63772] | 찌찌 | 이미
[TEST_63773] | 먼저 | 먼지
[TEST_63775] | 볶음 | 소음
[TEST_63779] | 총장 | 고장
[TEST_63782] | 대신하다 | 대단하다
[TEST_63787] | 전주 | 전부
[TEST_63792] | 덥다 | 쉽다
[TEST_63793] | 없다 | 어디
[TEST_63797] | 구십 | 수십
[TEST_63801] | 연습 | 연극
[TEST_63808] | 오로지 | 오렌지
[TEST_63814] | 육십 | 욕실
[TEST_63815] | 고음학교 | 고등학교
[TEST_63816] | 봉사 | 복사
[TEST_63817] | 너무 | 나무
[TEST_63818] | 대로 | 도로
[TEST_63823] | 소시지 | 메시지
[TEST_63830] | 발길 | 달걀
[TEST_63846] | 만족 | 가족
[TEST_63848] | 파파 | 하나
[TEST_63849] | 운문 | 운전
[TEST_63850] | 감도 | 강도
[TEST_63855] | 냄새 | 냄비
[TEST_63868] | 발끔 | 가끔
[TEST_63872] | 상인 | 장인
[TEST_63873] | 사사 | 사자
[TEST_63874] | 극장 

[TEST_66706] | 진동 | 지도
[TEST_66707] | 물돌 | 물론
[TEST_66713] | 그리다 | 그러다
[TEST_66718] | 불다 | 보다
[TEST_66723] | 국가적 | 국제적
[TEST_66724] | 대학 | 도착
[TEST_66728] | 파파 | 하나
[TEST_66743] | 그런데 | 그렇게
[TEST_66744] | 건강 | 긴장
[TEST_66745] | 힘쓰다 | 힘들다
[TEST_66746] | 비밀 | 비닐
[TEST_66747] | 등산 | 당신
[TEST_66748] | 횟수 | 호수
[TEST_66749] | 형식 | 현실
[TEST_66760] | 조멍 | 조명
[TEST_66761] | 박수 | 박스
[TEST_66767] | 이롭다 | 이러다
[TEST_66772] | 인체 | 인제
[TEST_66777] | 항의 | 회의
[TEST_66779] | 친정 | 긴장
[TEST_66787] | 앞문 | 아무
[TEST_66791] | 장사 | 장소
[TEST_66803] | 장미 | 자기
[TEST_66810] | 권전 | 완전
[TEST_66816] | 얼다 | 얼마
[TEST_66821] | 아빠 | 아마
[TEST_66823] | 그치다 | 그러다
[TEST_66826] | 파디 | 파리
[TEST_66829] | 수년 | 수면
[TEST_66830] | 목욕 | 목표
[TEST_66833] | 점말 | 정말
[TEST_66839] | 조기 | 저기
[TEST_66842] | 효도 | 하도
[TEST_66843] | 틀내 | 그래
[TEST_66851] | 이아가 | 이따가
[TEST_66852] | 외국 | 미국
[TEST_66855] | 와과 | 이거
[TEST_66857] | 가방 | 가장
[TEST_66859] | 전건 | 전선
[TEST_66864] | 의의 | 예의
[TEST_66868] | 아빠 | 아마
[TEST_66876] | 희망 | 사망
[TEST_66877] | 초밥 | 

[TEST_69666] | 딸기 | 빨리
[TEST_69667] | 오마 | 아마
[TEST_69670] | 운명 | 분명
[TEST_69675] | 운행 | 은행
[TEST_69676] | 반반 | 반찬
[TEST_69682] | 퐁풍 | 공중
[TEST_69686] | 서다 | 사다
[TEST_69696] | 성당 | 정당
[TEST_69700] | 신라 | 신사
[TEST_69703] | 열집 | 열정
[TEST_69705] | 마지막 | 하지만
[TEST_69706] | 환색 | 흰색
[TEST_69708] | 껍질 | 성질
[TEST_69709] | 최신 | 대신
[TEST_69719] | 잠자다 | 잠자리
[TEST_69724] | 기숙다 | 기숙사
[TEST_69728] | 코실 | 사실
[TEST_69732] | 나사 | 낫다
[TEST_69737] | 사이 | 아이
[TEST_69740] | 삼국 | 한국
[TEST_69757] | 마지막 | 하지만
[TEST_69761] | 꽃다 | 보다
[TEST_69768] | 살짝 | 사랑
[TEST_69770] | 열매 | 얼마
[TEST_69771] | 전개 | 전기
[TEST_69777] | 라면 | 남편
[TEST_69781] | 도서관 | 대사관
[TEST_69798] | 쏟다 | 보다
[TEST_69799] | 명명 | 명령
[TEST_69800] | 울이 | 우리
[TEST_69805] | 반지 | 단지
[TEST_69807] | 경치 | 경기
[TEST_69815] | 지렵다 | 어렵다
[TEST_69818] | 학생 | 항상
[TEST_69819] | 서류 | 서로
[TEST_69821] | 아흔 | 마흔
[TEST_69823] | 진단 | 집단
[TEST_69829] | 앉다 | 안다
[TEST_69830] | 약속 | 양복
[TEST_69831] | 여유 | 이유
[TEST_69836] | 병담 | 상담
[TEST_69840] | 비비 | 이미
[TEST_69843] | 지급 | 지금

[TEST_72701] | 조기 | 저기
[TEST_72710] | 진행 | 인생
[TEST_72713] | 어기다 | 여기다
[TEST_72714] | 신문 | 신분
[TEST_72715] | 막상 | 항상
[TEST_72722] | 열흘 | 역할
[TEST_72737] | 약품 | 작품
[TEST_72738] | 원하다 | 일하다
[TEST_72744] | 준비 | 전부
[TEST_72747] | 담배 | 자매
[TEST_72753] | 신문 | 신분
[TEST_72760] | 길다 | 있다
[TEST_72768] | 드라마 | 그나마
[TEST_72782] | 먼저 | 먼지
[TEST_72787] | 볶음 | 소음
[TEST_72789] | 껍질 | 성질
[TEST_72790] | 체험 | 시험
[TEST_72791] | 연간 | 연관
[TEST_72794] | 퐁종 | 공동
[TEST_72806] | 빼버 | 매번
[TEST_72812] | 명명 | 명령
[TEST_72814] | 수도 | 시도
[TEST_72815] | 중비 | 중부
[TEST_72821] | 거곳 | 그곳
[TEST_72825] | 전전 | 전선
[TEST_72841] | 봉다 | 보다
[TEST_72843] | 우선 | 어떤
[TEST_72847] | 편병 | 변명
[TEST_72850] | 막다 | 맞다
[TEST_72860] | 회화 | 회의
[TEST_72862] | 영국 | 연구
[TEST_72863] | 씻다 | 있다
[TEST_72866] | 갈색 | 발생
[TEST_72867] | 운동 | 온통
[TEST_72868] | 남남 | 남자
[TEST_72869] | 국회 | 교회
[TEST_72870] | 경경 | 공격
[TEST_72872] | 거액 | 거의
[TEST_72873] | 까까 | 아까
[TEST_72877] | 모인트 | 페인트
[TEST_72878] | 땅공 | 항공
[TEST_72879] | 선장 | 천장
[TEST_72885] | 살인 | 사진
[TE

In [33]:
not_in_corpus_cnt = 0
for index, row in sample_pred.iterrows():
    if row['label'] not in corpus:
        not_in_corpus_cnt += 1
print(f"Number of base prediction that not in the corpus: {not_in_corpus_cnt}")

Number of base prediction that not in the corpus: 39112
